In [1]:
import tensorflow as tf
from tensorflow.keras.layers import  Conv2D, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import SpatialDropout2D, Permute, Activation, Reshape, PReLU
from tensorflow.keras.layers import concatenate, add, Input
from tensorflow.keras.layers import BatchNormalization, MaxPooling2D
from tensorflow.keras.models import Model


In [2]:
__all__ = ['EENet']

def initial_block(inp, number_filter = 13, filter_size = (3,3), stride = (2,2)):
    
    conv = Conv2D(number_filter, filter_size, padding = "same", strides = stride ) (inp) 
    max_pool = MaxPooling2D() (inp)
    merged = concatenate ([conv,max_pool], axis = 3)
    batch = BatchNormalization(momentum = 0.1)(merged)  # enet_unpooling uses momentum of 0.1, keras default is 0.99
    output = PReLU(shared_axes = [1, 2])(batch)
    return output


In [3]:
def bottleneck(inp, number_filter = 32, internal_scale = 4, asymmetric = 0, dilated = 0, downsample = False, dropout_rate = 0.1):
    
    # main branch
    internal = number_filter // internal_scale
    encoder = inp
    
    # 1x1
    input_stride = 2 if downsample else 1  # the 1st 1x1 projection is replaced with a 2x2 convolution when downsampling
    encoder = Conv2D(internal,(input_stride, input_stride), (input_stride,input_stride) , use_bias =  False) (encoder)
    
    # Batch normalization + PReLU
    encoder = BatchNormalization(momentum = 0.1) (encoder)
    encoder = PReLU(shared_axes=[1, 2])(encoder)
    
    #con
    if not asymmetric and not dilated:
        print("both")
        encoder =  Conv2D(internal, (3,3),padding = "same" ) (encoder)
    elif asymmetric:
        print("as")
        encoder = Conv2D(internal, (1, asymmetric), use_bias = False, padding = "same") (encoder)
        encoder = Conv2D(internal, ( asymmetric, 1), padding = "same") (encoder)
    elif dilated:
        print("di")
        encoder = Conv2D(internal, (3,3), dilation_rate = (dilated, dilated), padding = "same") (encoder)
    
    # Batch normalization + PReLU
    encoder = BatchNormalization(momentum = 0.1) (encoder)
    encoder = PReLU(shared_axes=[1, 2])(encoder)
    
    return encoder

In [4]:
img = tf.random.uniform((1,256,256,3),)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [5]:
img1 = initial_block( img)   # #if  inpput = (1,256,256,3) output is TensorShape([1, 128, 128, 16])

NameError: name 'img' is not defined

In [139]:
img1.shape

TensorShape([1, 128, 128, 16])

In [140]:
img2 = bottleneck(img1, number_filter = 64, downsample = True)

both


In [141]:
img2.shape

TensorShape([1, 64, 64, 16])

In [142]:
encoder = Conv2D(16, (3, 3), dilation_rate=(5, 5), padding='same')(img2)

In [143]:
encoder.shape

TensorShape([1, 64, 64, 16])